In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''
engine = 'nous-16k'

In [3]:
def predict(messages):
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [4]:
system = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
system_malay = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant in malay language. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
next_step = 'Above is a conversation between a user and an intelligent assistant. Now suppose you are the user, generate response according to the generated material to continue the conversation.'

In [5]:
!mkdir ultrachat-lom-agc

mkdir: cannot create directory ‘ultrachat-lom-agc’: File exists


In [6]:
def ultrachat(row, n = 1):
    results = [
        {'role': 'context', 'content': row['paragraph']},
        {'role': 'user', 'content': row['question']},
    ]
    initial = f"""
    {row['paragraph']}

    {row['question']}
    """.strip()
    messages = [
        {'role': 'system', 'content': system_malay},
        {'role': 'user', 'content': initial},
    ]
    r = predict(messages)
    results.append({
        'role': 'assistant', 'content': r,
    })
    messages.append({
        'role': 'assistant', 'content': r,
    })
    
    for _ in range(n):
    
        messages_temp = messages + [
            {'role': 'user', 'content': 'Now suppose you are the user, say something to continue the conversation based on given context. Make the response short and the language casual'}
        ]
        r2 = predict(messages_temp)
        if r2 is None:
            break

        results.append({
            'role': 'user', 'content': r2,
        })
        messages.append({
            'role': 'user', 'content': r2,
        })

        r = predict(messages)
        if r is None:
            break
        results.append({
            'role': 'assistant', 'content': r,
        })
        messages.append({
            'role': 'assistant', 'content': r,
        })
    
    return results

In [7]:
from glob import glob

files = sorted(glob('question-lom-agc/*.json'))
len(files)

8320

In [8]:
def generate(f):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = f'ultrachat-lom-agc/{index}.json'
    if os.path.exists(filename):
        return
        
    with open(f) as fopen:
        data = json.load(fopen)
    
    if len(data['paragraph'].split()) < 100:
        return
    
    r = ultrachat(data, n = random.randint(3, 7))
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

In [9]:
generate(files[-3])

In [10]:
with open('ultrachat-lom-agc/997.json') as fopen:
    data = json.load(fopen)
    
data

[{'role': 'context',
  'content': '\n\nPindaan peraturan 37\n\n24. Peraturan 37 Peraturan-Peraturan ibu dipinda dengan menggantikan perkataan\n\n“dalam Jadual I dan disertakan dengan bayaran sebanyak dua ratus ringgit dalam\n\nbentuk kiriman wang, draf bank, cek atau apa-apa bentuk lain sebagaimana yang\n\nditentukan oleh Lembaga, yang kena dibayar kepada Lembaga” dengan perkataan\n\n“di laman sesawang Lembaga dan disertakan dengan suatu fi untuk Peperiksaan\n\nPenilaian Professional sebagaimana yang dinyatakan dalam Jadual III.”.\n\n\n\nBahagian baru VIA\n\n25. Peraturan-Peraturan ibu dipinda dengan memasukkan selepas Bahagian VI\n\nbahagian  yang berikut:   Peperiksaan\n\nKompetensi\n\nProfesional.\n\n“BAHAGIAN  VIA\n\nPEPERIKSAAN  KOMPETENSI PROFESIONAL\n\n\n\n38A. (1)     Jurutera Profesional yang berhasrat untuk berdaftar sebagai\n\nJurutera  Profesional dengan Perakuan Amalan di bawah seksyen 10D\n\nAkta, hendaklah lulus Peperiksaan Kompetensi Profesional.\n\n(2)  Tiap-tiap Jurut

In [11]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range(0, len(files), max_worker)):
    b = files[i: i + max_worker]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f): f for f in b}

        for future in as_completed(futures):
            future.result()

 25%|█████████                           | 1045/4160 [17:32<35:14:34, 40.73s/it]

Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 27%|█████████▋                          | 1122/4160 [59:50<28:11:34, 33.41s/it]

Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 41%|█████████████▊                    | 1690/4160 [5:52:31<16:36:03, 24.20s/it]

...

 96%|████████████████████████████████▍ | 3974/4160 [23:08:29<1:53:46, 36.70s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
Invalid value for 'content': expected a string, got null.


 96%|████████████████████████████████▋ | 4002/4160 [23:22:22<1:17:22, 29.38s/it]

'content'


100%|████████████████████████████████████| 4160/4160 [24:39:18<00:00, 21.34s/it]
